In [219]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from bs4 import BeautifulSoup
from utils import *

path_2004 = "./errors/"
all_2004 = os.listdir(path_2004)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [232]:
results = []
for i, fname in enumerate(all_2004):
    print("File: ", i)
    test_f = open(path_2004 + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1]  

    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    rows = []
    if table:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
    else:
        lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
            multiplier = get_multiplier_from_tbl_list(tbl_list)

            
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                   '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category'], 'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier}

        results.append(result)
pd.DataFrame(results).to_csv('errors_df')

File:  0
No HTML tables!
File:  1


/Users/divyas/Documents/G1/sec/utils.py:179: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 179 of the file /Users/divyas/Documents/G1/sec/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(wholetext)


Found a table
File:  2
No contractual tables!
File:  3
No HTML tables!
File:  4
No HTML tables!
Returning:  []
File:  5
No HTML tables!
File:  6
No contractual tables!
File:  7
No HTML tables!
Returning:  []
File:  8
No HTML tables!
File:  9
No HTML tables!
Unknown error:  list index out of range
File:  10
No HTML tables!
Returning:  []
File:  11
No HTML tables!
Returning:  []
File:  12
No HTML tables!
Unknown error:  list index out of range
File:  13
No contractual tables!
Returning:  []
File:  14
No HTML tables!
File:  15
Found a table
File:  16
No HTML tables!
Returning:  []
File:  17
No HTML tables!
Unknown error:  list index out of range
File:  18
No HTML tables!
File:  19
No HTML tables!
File:  20
No HTML tables!
File:  21
No HTML tables!
Unknown error:  list index out of range
File:  22
No HTML tables!
File:  23
No HTML tables!
File:  24
No HTML tables!
Returning:  []
File:  25
No contractual tables!
Unknown error:  list index out of range
File:  26
No HTML tables!
File:  27
No 

In [233]:
results_df = pd.DataFrame(results)
#results_df = pd.read_csv('scraped_files_df')
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print(len(set(last_check['fname'])), len(set(row_check['fname'])), len(set(none_check['fname'])))

3 11 14


In [239]:
row_check

,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
5,-,-,-,-,NaN,0000077597,20021102,0000077597-03-000003.txt,1,0,1,-
8,-,-,-,-,NaN,0000054291,20021231,0000054291-03-000003.txt,1,0,1,-
10,-,-,-,-,NaN,0000040570,20030930,0000040570-03-000009.txt,1000,0,1,-
11,-,-,-,-,NaN,0000077543,20021231,0000077543-03-000006.txt,1,0,1,-
12,-,-,-,-,NaN,0000015486,20021228,0000015486-03-000002.txt,1,0,1,-
13,-,-,-,-,NaN,0000015486,20050101,0000015486-05-000004.txt,1,0,1,-
14,-,-,-,-,NaN,0000037914,20021102,0000037914-03-000002.txt,1,0,1,-
19,-,-,-,-,NaN,0000009389,20021231,0000009389-03-000044.txt,1,0,1,-
20,-,-,-,-,NaN,0000051303,20031031,0000051303-03-000014.txt,1,0,1,-
24,-,-,-,-,NaN,0000022551,20021231,0000022551-03-000001.txt,1,0,1,-


# Examining one file

In [218]:
path_prefix = "./2005/"
test_f = open(path_prefix + "0001104659-05-013857.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()
#lines, tbl_list = scrape_text_table(wholetext)
#scores, tables = scrape_table(wholetext)
table = scrape_table(wholetext)
rows = scrape_rows(table)
rows

CIK:  0001099291 Date:  20041231
Found a table
['($000\x92s)', 'total', '2005', '2006-2007', '2008-2009', '2010 &\n  thereafter', 'commitments\n  reference footnote\n  reference']
['debt service\n  obligations (including interest) $', '617,295', '$', '32,963', '$', '66,197', '$', '67,473', '$', '450,662', '(a)', '(5)']
['operations\n  agreement', '142,781', '5,358', '11,128', '11,634', '114,661', '(b)', '(8)']
['interconnection\n  agreement', '4,708', '214', '428', '428', '3,638', '(c)', '(8)']
['total $', '764,784', '$', '38,535', '$', '77,753', '$', '79,535', '$', '568,961']
HEADER:  ['($000\x92s)', 'total', '2005', '2006-2007', '2008-2009', '2010 &\n  thereafter', 'commitments\n  reference footnote\n  reference']
total idx:  1
Row:  ['operations\n  agreement', '142781', '5358', '11128', '11634', '114661', '(b)', '(8)']
Row length:  8 7
Unknown error 1:  could not convert string to float: '(b)' -- processing as longer row
hdrs ['($000\x92s)', 'total', '2005', '2006-2007', '2008-2009'

[{'1-3': '11128',
  '3-5': '11634',
  '<1': '5358',
  '>5': '114661',
  'category': 'operations\n  agreement',
  'total': 142781.0}]

In [209]:
#table_to_list(table)
table_to_list(tables[np.where(scores == 4)[0][1]])
#scores

IndexError: index 1 is out of bounds for axis 0 with size 1

In [134]:
lt = [table for table in tables if ('liabilities reflected in balance sheet' in table.text)]

In [135]:
#lt[1].text.replace('\n', '').replace('\xa0', '')
len(lt)

1